In [ ]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Using GPT Tree Index

#### [Demo] Default leaf traversal 

In [ ]:
from llama_index import GPTTreeIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [2]:
documents = SimpleDirectoryReader('data').load_data()

In [ ]:
new_index = GPTTreeIndex.from_documents(documents)

In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = new_index.as_query_engine()
response = query_engine.query("What did the author do growing up?")

In [4]:
display(Markdown(f"<b>{response}</b>"))

<b>The author wrote short stories and tried to program on an IBM 1401.</b>

In [ ]:
# set Logging to DEBUG for more detailed outputs
response = query_engine.query("What did the author do after his time at Y Combinator?")

In [6]:
display(Markdown(f"<b>{response}</b>"))

<b>The author went on to start his own company.</b>

#### [Demo] Leaf traversal with child_branch_factor=2

In [ ]:
# try using branching factor 2
query_engine = new_index.as_query_engine(
    child_branch_factor=2
)
response = query_engine.query("What did the author do growing up?")

In [6]:
display(Markdown(f"<b>{response}</b>"))

<b>The author grew up writing simple programs on a TRS-80 computer, as well as trying to program on an IBM 1401. In the early 1990s, the author was a student at the Rhode Island School of Design (RISD) and then the Accademia di Belle Arti in Florence, Italy. They eventually dropped out of RISD and moved to New York City, where they got a job at Interleaf, a software company. While working there, they learned about a new markup language called HTML, which would later become a big part of their life. He also wrote a book on Lisp programming.</b>

#### [Demo] Build Tree Index during Query-Time

In [3]:
documents = SimpleDirectoryReader('data').load_data()

In [4]:
index_light = GPTTreeIndex.from_documents(documents, build_tree=False)

> [build_index_from_documents] Total token usage: 0 tokens


In [7]:
query_engine = index_light.as_query_engine(
    retriever_mode="all_leaf",
    response_mode='tree_summarize',
)
query_engine.query("What did the author do after his time at Y Combinator?")

> Starting query: What did the author do after his time at Y Combinator?
> Building index from nodes: 5 chunks
0/57
10/57
20/57
30/57
40/57
50/57
> [query] Total token usage: 18200 tokens


'\nThe author went back to painting.'

#### [Demo] Build Tree Index with a custom Summary Prompt, directly retrieve answer from root node

In [20]:
from llama_index import Prompt

In [23]:
documents = SimpleDirectoryReader('data').load_data()

query_str = "What did the author do growing up?"
SUMMARY_PROMPT_TMPL = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    f"answer the question: {query_str}\n"
)
SUMMARY_PROMPT = Prompt(SUMMARY_PROMPT_TMPL)
index_with_query = GPTTreeIndex.from_documents(documents, summary_template=SUMMARY_PROMPT)

> Building index from nodes: 5 chunks
0/57
10/57
20/57
30/57
40/57
50/57
> [build_index_from_documents] Total token usage: 18031 tokens


In [ ]:
# directly retrieve response from root nodes instead of traversing tree
query_engine = index_with_query.as_query_engine(
    retriever_mode="root"
)
response = index_with_query.query(query_str)

In [10]:
display(Markdown(f"<b>{response}</b>"))

<b>
The author was homeschooled and then attended a prestigious art school. The author grew up writing essays and thinking about other things he could work on.</b>

## Using GPT Keyword Table Index

In [7]:
from llama_index import GPTKeywordTableIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [2]:
# build keyword index
documents = SimpleDirectoryReader('data').load_data()
index = GPTKeywordTableIndex.from_documents(documents)

> Processing chunk 0 of 6: 		

What I Worked On

February 2021

Before col...
> Keywords: ['painting', 'computers', 'programming', 'lisp', 'ai', 'college', 'graduate school', 'graduate', 'school', 'writing']
> Processing chunk 1 of 6: of excluding them, because there were so many s...
> Keywords: ['school', 'students', 'painting', 'florence', 'risd', 'accademia', 'still lives', 'still', 'lives', 'color', 'new york', 'new', 'york', 'yorkville', 'idelle weber', 'idelle', 'weber', 'harvard', 'world wide web', 'world', 'wide', 'web', 'y combinator', 'combinator', 'software', 'lisp']
> Processing chunk 2 of 6: an alarming prospect, because neither of us kne...
> Keywords: ['windows', 'unix', 'lisp', 'web app', 'web', 'app', 'browser', 'store builder', 'store', 'builder', 'ecommerce', 'startup', 'painting']
> Processing chunk 3 of 6: browser, and then host the resulting applicatio...
> Keywords: ['y combinator', 'combinator', 'investment', 'summer founders program', 'summer', 'founders', 'pr

In [9]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do after his time at Y Combinator?")

> Starting query: What did the author do after his time at Y Combinator?
Extracted keywords: ['y combinator', 'combinator']
> Querying with idx: 7143669651211954504: of excluding them, because there were so many s...
> Querying with idx: 4978118451876167434: browser, and then host the resulting applicatio...
> Querying with idx: 7378313280237489139: person, and from those we picked 8 to fund. The...
> Querying with idx: 2670584622494666310: it was like living in another country, and sinc...


In [10]:
display(Markdown(f"<b>{response}</b>"))

<b>

After a few years, the author decided to step away from Y Combinator to focus on other projects, such as painting and writing essays. In 2013, he handed over control of Y Combinator to Sam Altman. The author's mother passed away in 2014, and after taking some time to grieve, he returned to writing essays and working on Lisp. He continued working on Lisp until 2019, when he finally completed the project.

In 2015, the author decided to move to England with his family. They originally intended to only stay for a year, but ended up liking it so much that they remained there. The author wrote Bel while living in England. In 2019, he finally finished the project. After completing Bel, the author wrote a number of essays on various topics. He continued writing essays through 2020, but also started thinking about other things he could work on.</b>

## Using GPT List Index

In [ ]:
from llama_index import GPTListIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [2]:
# build list index
documents = SimpleDirectoryReader('data').load_data()
index = GPTListIndex.from_documents(documents)

> Adding chunk: 		

What I Worked On

February 2021

Before col...
> Adding chunk: only up to age 25 and already there are such co...
> Adding chunk: clear that it was even possible. To find out, w...
> Adding chunk: a name for the kind of company Viaweb was, an "...
> Adding chunk: get their initial set of customers almost entir...
> Adding chunk: had smart people and built impressive technolog...
> [build_index_from_documents] Total token usage: 0 tokens


In [ ]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do after his time at Y Combinator?")

In [16]:
display(Markdown(f"<b>{response}</b>"))

<b>

After his time at Y Combinator, the author moved back to Providence to continue at RISD. However, he found that art school was not what he expected it to be and dropped out. He then moved to New York City and started writing a book on Lisp. When that didn't work out, he started a company to put art galleries online. However, that also failed. He then had the idea to start a company to build online stores, which became a success.

The author then left his position at Yahoo to pursue painting full-time. However, he found it difficult to get back into the painting mindset and eventually returned to New York City. It was there that he had the idea to create a web application that would allow users to create and host their own web applications.</b>